# 导包，设置根目录

In [13]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import random
import pickle
import multiprocessing
import re
import pickle

from collections import Counter
from itertools import chain
import datetime

# 更改工作目录为当前项目根目录
import sys
import os
os.chdir(sys.path[0])
print(os.getcwd())

C:\workfile\python\Log-diagnosis


In [16]:
word_df = pd.read_csv('pre_contest/dataset/word_label_df.txt',sep='\t',index_col=0)
word_df.groupby('label').size()

label
0    1460
1    3016
2    7731
3    2214
dtype: int64

In [17]:
log_str_list=list(word_df['log'])
label_list=list(word_df['label'])

In [7]:
word_df.head()

,log,label
0,management subsys health system_health | sens...,1
1,memory cpu1d0_dimm_stat | correctable ecc | a...,2
2,processor cpu0_status | configuration error |...,1
3,processor cpu0_status | configuration error |...,2
4,drive slot hdd_l_14_status | drive fault | de...,3


# 分词及词频（仅保留分词后的字母数字下划线）统计，每条日志的词频向量

In [18]:
# 统计词频
word_lists = [word_tokenize(ith) for ith in log_str_list]
word_list=[re.sub(r'[^\w]','',str(xx)).lower() for xx in list(chain(*word_lists))]
word_frequency={}
for word in word_list:
    if word:
        if word in word_frequency:
            word_frequency[word]+=1
        else:
            word_frequency[word]=1
word_frequency=dict(sorted(word_frequency.items(),key=lambda item:item[1],reverse=True))
word_frequency_df=pd.DataFrame({'word':list(word_frequency.keys()),'frequency':list(word_frequency.values())})
# word_frequency_df.to_csv('pre_contest/dataset/word_frequency_df.txt',sep='\t')

In [111]:
# 计算每条日志的词频向量
frequency_vector_list = []
tag=0
for word in word_frequency:
    if tag%100==0:
        print(tag,datetime.datetime.now())
    frequency_vector = [sum(np.array([re.sub(r'[^\w]','',str(item)).lower() for item in word_lists[xth]]) == word) for xth in range(len(word_lists))]
    frequency_vector_list.append(frequency_vector)
    tag+=1


0 2022-03-13 19:52:27.748411
100 2022-03-13 19:56:18.240527
200 2022-03-13 20:00:08.692700
300 2022-03-13 20:03:58.898712
400 2022-03-13 20:07:50.106271
500 2022-03-13 20:11:42.065111
600 2022-03-13 20:15:33.612238
700 2022-03-13 20:19:26.893161
800 2022-03-13 20:23:19.930044
900 2022-03-13 20:27:13.038312
1000 2022-03-13 20:31:06.576821
1100 2022-03-13 20:34:59.606510
1200 2022-03-13 20:38:52.404158
1300 2022-03-13 20:42:45.311422
1400 2022-03-13 20:46:38.944049
1500 2022-03-13 20:50:29.874680
1600 2022-03-13 20:54:20.902964


In [166]:
frequency_vector_df=pd.DataFrame(frequency_vector_list)
frequency_vector_df=frequency_vector_df.T
frequency_vector_df.columns=list(word_frequency.keys())
frequency_vector_df['label']=label_list
frequency_vector_df['sn']=word_df['sn']
frequency_vector_df['day']=word_df['day']
# frequency_vector_df.to_csv('pre_contest/dataset/frequency_vector_df.txt',sep='\t')

In [165]:
frequency_vector_df

,asserted,power,system,memory,drive,initiated,deasserted,processor,boot,state,...,dimm031_status,nic13,dimm171_status,9120,dimm1b,memory_hot,00fc92f15f00,label,sn,day
0,6,2,3,0,0,2,3,4,1,2,...,0,0,0,0,0,0,0,1,SERVER_10001,2020-05-01
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,SERVER_10003,2020-03-28
2,3,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,1,SERVER_10008,2020-02-25
3,4,1,1,1,0,2,0,2,1,0,...,0,0,0,0,0,0,0,2,SERVER_10008,2020-03-11
4,2,0,0,0,8,0,2,0,0,0,...,0,0,0,0,0,0,0,3,SERVER_10009,2020-05-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14416,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,SERVER_9991,2020-08-04
14417,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,SERVER_9991,2020-10-07
14418,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,SERVER_9993,2020-05-14
14419,2,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,SERVER_9998,2020-05-29


# 训练XGB模型

In [135]:
random.seed(0)
feature=np.array(frequency_vector_list).T
label=np.array(label_list)
val_mask = [random.random() < 0.2 for _ in range(len(feature))]
train_mask = [not xx for xx in val_mask]
val_feature = feature[val_mask]
val_label = label[val_mask]
train_feature = feature[train_mask]
train_label = label[train_mask]

In [136]:
train_data=xgb.DMatrix(train_feature,label=train_label)
train_feature=xgb.DMatrix(train_feature)
val_feature=xgb.DMatrix(val_feature)
# xgb模型参数
params = {
    'booster':'gbtree',
    'objective':'multi:softmax',   # 多分类问题
    'num_class':4,  # 类别数，与multi softmax并用
    'gamma':0.1,    # 用于控制是否后剪枝的参数，越大越保守，一般0.1 0.2的样子
    'max_depth':6,  # 构建树的深度，越大越容易过拟合
    'lambda':2,  # 控制模型复杂度的权重值的L2 正则化项参数，参数越大，模型越不容易过拟合
    'subsample':1, # 随机采样训练样本
    'colsample_bytree':1,# 这个参数默认为1，是每个叶子里面h的和至少是多少
    # 对于正负样本不均衡时的0-1分类而言，假设h在0.01附近，min_child_weight为1
    #意味着叶子节点中最少需要包含100个样本。这个参数非常影响结果，
    # 控制叶子节点中二阶导的和的最小值，该参数值越小，越容易过拟合
    'silent':0,  # 设置成1 则没有运行信息输入，最好是设置成0
    'eta':0.3,  # 如同学习率
    'seed':1000,
    'nthread':12,  #CPU线程数
    #'eval_metric':'auc'
}
xgb_model=xgb.train(params,train_data,num_boost_round=500)

[21:11:25] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:11:25] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [137]:
train_pred=xgb_model.predict(train_feature)
val_pred=xgb_model.predict(val_feature)

In [139]:
# 指标评估
def macro_f1(label,prediction)  -> float:

    """
    计算得分
    :param target_df: [sn,fault_time,label]
    :param submit_df: [sn,fault_time,label]
    :return:
    """

    weights =  [3  /  7,  2  /  7,  1  /  7,  1  /  7]
    macro_F1 =  0.
    for i in  range(len(weights)):
        TP =  np.sum((label==i) & (prediction==i))
        FP =  np.sum((label!= i) & (prediction == i))
        FN =  np.sum((label == i) & (prediction!= i))
        precision = TP /  (TP + FP)  if  (TP + FP)  >  0  else  0
        recall = TP /  (TP + FN)  if  (TP + FN)  >  0  else  0
        F1 =  2  * precision * recall /  (precision + recall)  if  (precision + recall)  >  0  else  0
        macro_F1 += weights[i]  * F1
        
        print('Task %d:\n Prcesion %.2f, Recall %.2f, F1 %.2f' % (i+1, precision, recall, F1))
        
    return macro_F1

In [140]:
macro_f1(train_label,train_pred)

Task 1:
 Prcesion 0.92, Recall 0.49, F1 0.64
Task 2:
 Prcesion 0.81, Recall 0.94, F1 0.87
Task 3:
 Prcesion 0.98, Recall 0.99, F1 0.98
Task 4:
 Prcesion 0.93, Recall 0.98, F1 0.95


0.7989989625547163

In [141]:
macro_f1(val_label,val_pred)

Task 1:
 Prcesion 0.56, Recall 0.24, F1 0.34
Task 2:
 Prcesion 0.71, Recall 0.84, F1 0.77
Task 3:
 Prcesion 0.95, Recall 0.96, F1 0.96
Task 4:
 Prcesion 0.88, Recall 0.95, F1 0.91


0.6311281891972205

# 对初赛测试集进行预测

In [19]:
preliminary_submit_dataset=pd.read_csv('./pre_contest/dataset/preliminary_submit_dataset.txt',sep='\t')
test_log_str_list=list(preliminary_submit_dataset['log'])
test_word_lists = [word_tokenize(str(ith)) for ith in test_log_str_list]
test_frequency_vector_list = []
tag=0
for word in word_frequency:
    if tag%100==0:
        print(tag,datetime.datetime.now())
    test_frequency_vector = [sum(np.array([re.sub(r'[^\w]','',str(item)).lower() for item in test_word_lists[xth]]) == word) for xth in range(len(test_word_lists))]
    test_frequency_vector_list.append(test_frequency_vector)
    tag+=1


0 2022-03-15 00:14:59.990387
100 2022-03-15 00:15:20.008418
200 2022-03-15 00:15:40.003317
300 2022-03-15 00:15:59.992554
400 2022-03-15 00:16:19.983751
500 2022-03-15 00:16:39.992823
600 2022-03-15 00:17:00.068920
700 2022-03-15 00:17:20.107527
800 2022-03-15 00:17:40.166019
900 2022-03-15 00:18:00.253310
1000 2022-03-15 00:18:20.299403
1100 2022-03-15 00:18:40.293634
1200 2022-03-15 00:19:00.275526
1300 2022-03-15 00:19:20.286810
1400 2022-03-15 00:19:40.246421
1500 2022-03-15 00:20:00.237720
1600 2022-03-15 00:20:20.252362


In [20]:
test_frequency_vector_df=pd.DataFrame(test_frequency_vector_list)
test_frequency_vector_df=test_frequency_vector_df.T
test_frequency_vector_df.columns=list(word_frequency.keys())
test_frequency_vector_df[['sn','fault_time']]=preliminary_submit_dataset[['sn','fault_time']]
# test_frequency_vector_df.to_csv('pre_contest/dataset/test_frequency_vector_df.txt',sep='\t')

In [168]:
test_frequency_vector_df

,asserted,power,system,memory,drive,initiated,deasserted,processor,boot,state,...,nic11,dimm031_status,nic13,dimm171_status,9120,dimm1b,memory_hot,00fc92f15f00,sn,fault_time
0,2,1,2,0,0,4,0,0,2,0,...,0,0,0,0,0,0,0,0,000d33b21436,2020-09-02 16:42:54
1,4,2,3,1,0,2,0,0,1,2,...,0,0,0,0,0,0,0,0,005c5a9218ba,2020-06-28 19:05:16
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0079283bde6e,2020-04-26 21:32:44
3,5,2,3,1,0,2,0,0,1,2,...,0,0,0,0,0,0,0,0,007bdf23b62f,2020-06-16 18:40:39
4,6,0,3,2,0,1,0,0,3,0,...,0,0,0,0,0,0,0,0,00a577a8e54f,2020-04-07 07:16:55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3006,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ffbf46b4af21,2019-12-28 20:10:01
3007,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ffc229b6cd9a,2020-06-27 02:39:08
3008,2,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,ffd44698a52b,2020-01-21 15:46:56
3009,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,fff73a9e5bd5,2020-03-01 22:43:43


In [185]:
test_frequency_vector_df=pd.read_csv('pre_contest/dataset/test_frequency_vector_df.txt',sep='\t')
train_data=xgb.DMatrix(feature,label=label)
xgb_model_V1=xgb.train(params,train_data,num_boost_round=500)
test_feature_temp=np.array(test_frequency_vector_list).T
test_feature=xgb.DMatrix(test_feature_temp)
preliminary_submit_dataset['prediction']=xgb_model.predict(test_feature)

[21:57:21] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:57:21] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [209]:
preliminary_submit_dataset_a=preliminary_submit_dataset[['sn','fault_time','prediction']]
preliminary_submit_dataset_a['label']=preliminary_submit_dataset_a['prediction'].apply(lambda x : int(x))
preliminary_submit_dataset_a=preliminary_submit_dataset_a[['sn','fault_time','label']]
preliminary_submit_dataset_a.to_csv('pre_contest/dataset/preliminary_submit_dataset_a_V1.csv',sep=',',index=None)

C:\Users\jinca\AppData\Local\Temp\ipykernel_2032\2213533351.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preliminary_submit_dataset_a['label']=preliminary_submit_dataset_a['prediction'].apply(lambda x : int(x))


In [210]:
preliminary_submit_dataset_a

,sn,fault_time,label
0,000d33b21436,2020-09-02 16:42:54,3
1,005c5a9218ba,2020-06-28 19:05:16,2
2,0079283bde6e,2020-04-26 21:32:44,3
3,007bdf23b62f,2020-06-16 18:40:39,2
4,00a577a8e54f,2020-04-07 07:16:55,2
...,...,...,...
3006,ffbf46b4af21,2019-12-28 20:10:01,2
3007,ffc229b6cd9a,2020-06-27 02:39:08,2
3008,ffd44698a52b,2020-01-21 15:46:56,2
3009,fff73a9e5bd5,2020-03-01 22:43:43,2
